<a href="https://colab.research.google.com/github/VVasanth/ReinforcementLearning/blob/main/TicTacToe_TFAgentTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "gym>=0.21.0"
!pip install tf-agents

!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 696 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.24.1-py3-none-any.whl size=793155 sha256=7115350d9dbc944c22befa5852135177c8b6a7efd94e7630a9da77ba27a8b120
  Stored in directory: /root/.cache/pip/wheels/18/0e/54/63d9f3d16ddf0fec1622e90d28140df5e6016bcf8ea920037d
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.5 MB/s 
     |████████████████████████████████| 18.3 MB 122 kB/s 
     |████████████████████████████████| 624 kB 66.5 MB/s 
  Installing build dependen

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
import numpy as np
import random
from itertools import product

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [ ]:
import itertools

from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
import numpy as np
import random
from itertools import product

class TicTacToeAgentEnv(py_environment.PyEnvironment):

    def __init__(self, name):
        self.board_rows = 3
        self.board_cols = 3
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=8, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(9,), dtype=np.float64, name='observation')
        self._step_counter = 0
        self.board = np.zeros((self.board_rows, self.board_cols))
        self.name = name
        self.lr = 0.2
        self.exp_rate = 0.3
        self.decay_gamma = 0.9
        self.episode_ended = False
        self.base_positions = self.generatePositions(self.board_rows)
        self._no_of_tries = 10
        self._states = np.zeros(self.board_rows * self.board_cols)

    def generatePositions(self, size):
        # initialize N
        # All possible N combination tuples
        # Using list comprehension + product()
        res = [ele for ele in product(range(0, size), repeat=size-1)]
        return res

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def get_obs(self):
        return self._states

    def get_availablePositions(self):
        positions = []
        for i in range(self.board_rows):
            for j in range(self.board_cols):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

    def _reset(self):
        self.board = np.zeros((self.board_rows, self.board_cols))
        self._states = np.zeros(self.board_rows*self.board_cols)
        self.episode_ended = False
        self._step_counter = 0
        return ts.restart(self.get_obs())

    def updateState(self,action, position, value):
        self._states[action] = 1
        self.board[position] = value
        status = self.checkGameStatus()
        return status

    def checkGameStatus(self):
        # row
        for i in range(self.board_rows):
            if sum(self.board[i, :]) == 3:
                self.episode_ended = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.episode_ended = True
                return -1
        # col
        for i in range(self.board_cols):
            if sum(self.board[:, i]) == 3:
                self.episode_ended = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.episode_ended = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(self.board_cols)])
        diag_sum2 = sum([self.board[i, self.board_cols - i - 1] for i in range(self.board_cols)])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:
            self.episode_ended = True
            return 1
        if diag_sum == -3:
            self.episode_ended = True
            return -1

        # tie
        # no available positions
        if len(self.get_availablePositions()) == 0:
            self.episode_ended = True
            return -0.8
        # not end
        self.episode_ended = False
        return None

    def checkPositionExists(self, position, avlPositions):
        retValue = False
        for i in range(len(avlPositions)):
            posVal = avlPositions[i]
            if (posVal == position):
                retValue = True
                return retValue
        return retValue

    def _step(self, action):
        updateStateFlag = 0

        avl_positions = self.get_availablePositions()
        if (len(avl_positions) == 0):
            self.episode_ended = True

        if self.episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            #print("# of steps performed : " + str(self._step_counter))
            self._reset()

        position = self.base_positions[action]
        if(self._step_counter==self._no_of_tries):
            self.episode_ended = True
            reward = -1
            return ts.termination(self.get_obs(), reward)
        self._step_counter = self._step_counter + 1


        if self.checkPositionExists(position, avl_positions):
            gameStatus = self.updateState(action, position, 1)
            if (gameStatus == -1):
                reward = -1
                self.episode_ended = True
                return ts.termination(self.get_obs(), reward)
            elif (gameStatus == 1):
                reward = 1
                self.episode_ended = True
                return ts.termination(self.get_obs(), reward)
            elif (gameStatus == -0.8):
                reward = -0.8
                self.episode_ended = True
                return ts.termination(self.get_obs(), reward)
            elif (gameStatus == None):
                reward = 0.1
                avlPositions = self.get_availablePositions()
                oppPosition = random.choice(avlPositions)
                oppAction = self.base_positions.index(oppPosition)
                gameStatus = self.updateState(oppAction,oppPosition, -1)
                if (gameStatus == -1):
                    reward = -1
                    self.episode_ended = True
                    return ts.termination(self.get_obs(), reward=reward)
                else:
                    return ts.transition(self.get_obs(), reward=reward, discount=1.0)
        else:
            
            #print("chosen non-existent action at the step " + str(self._step_counter))
            reward = -1
            return ts.termination(self.get_obs(), reward=reward)
            #return ts.transition(self.get_obs(), reward=reward, discount=1.0)


In [ ]:
tictacplayer1 = TicTacToeAgentEnv("p1")

In [ ]:
from tf_agents.environments import utils
utils.validate_py_environment(tictacplayer1, episodes=15)

In [ ]:
tictacplayer2 = TicTacToeAgentEnv("p2")
utils.validate_py_environment(tictacplayer2, episodes=15)

In [ ]:
from tf_agents.environments import tf_py_environment

#train_py_env = suite_gym.wrap_env(train_qt_env)
tictacplayer1_tf = tf_py_environment.TFPyEnvironment(tictacplayer1)

#eval_py_env = suite_gym.wrap_env(eval_qt_env)
tictacplayer2_tf = tf_py_environment.TFPyEnvironment(tictacplayer2)

In [ ]:
from tf_agents.policies import random_tf_policy
random_policy = random_tf_policy.RandomTFPolicy(tictacplayer1_tf.time_step_spec(), tictacplayer1_tf.action_spec())

In [ ]:
time_step = tictacplayer1_tf._reset()

In [ ]:
time_step

TimeStep(
{'discount': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 'observation': <tf.Tensor: shape=(1, 9), dtype=float64, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])>,
 'reward': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>})

In [ ]:
action_step = random_policy.action(time_step)

In [ ]:
action_step

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, state=(), info=())

In [ ]:
learning_rate = 1e-3 
num_eval_episodes = 10
replay_buffer_max_length = 100000

In [ ]:
import tensorflow as tf
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.networks import sequential

fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(tictacplayer1_tf.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [ ]:
num_actions

9

In [ ]:
from tf_agents.agents.dqn import dqn_agent

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tictacplayer1_tf.time_step_spec(),
    tictacplayer1_tf.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(tictacplayer1_tf.time_step_spec(), tictacplayer1_tf.action_spec())

In [ ]:
def compute_avg_return(environment, policy, num_episodes=5):

  total_wins = 0
  total_loss = 0
  total_ties = 0
  no_of_steps_in_win = 0
  no_of_steps_in_loss = 0
  avg_steps_in_win = 0
  avg_steps_in_loss = 0

  for _ in range(num_episodes):
    time_step = environment._reset()
    no_of_steps = 0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      no_of_steps = no_of_steps + 1
      time_step = environment._step(action_step.action)
      if(time_step.reward==1):
        total_wins = total_wins + 1
        no_of_steps_in_win = no_of_steps_in_win + no_of_steps
      elif(time_step.reward==-1):
        total_loss = total_loss + 1
        no_of_steps_in_loss = no_of_steps_in_loss + no_of_steps
      elif(time_step.reward==-0.8):
        total_ties = total_ties + 1

  if(total_wins!=0):
    avg_steps_in_win = no_of_steps_in_win/total_wins
  
  if(total_loss!=0):
    avg_steps_in_loss = no_of_steps_in_loss/total_loss

  return total_wins, total_loss, total_ties, avg_steps_in_win, avg_steps_in_loss

In [ ]:
compute_avg_return(tictacplayer2_tf, random_policy, num_episodes=20)

(5, 15, 0, 3.6, 3.066666666666667)

In [ ]:
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver

In [ ]:
replay_buffer = TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tictacplayer1_tf.batch_size,
    max_length=100_000)

In [ ]:
tictacplayer1_tf._reset()

init_driver = DynamicStepDriver(
    tictacplayer1_tf,
    random_policy,
    observers=[replay_buffer.add_batch],
    num_steps=2_500)
final_time_step, final_policy_state = init_driver.run()

In [ ]:
replay_buffer.gather_all()

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.


Trajectory(
{'action': <tf.Tensor: shape=(1, 6488), dtype=int32, numpy=array([[0, 2, 3, ..., 3, 4, 6]], dtype=int32)>,
 'discount': <tf.Tensor: shape=(1, 6488), dtype=float32, numpy=array([[1., 1., 0., ..., 0., 1., 0.]], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(1, 6488), dtype=int32, numpy=array([[1, 1, 2, ..., 2, 1, 2]], dtype=int32)>,
 'observation': <tf.Tensor: shape=(1, 6488, 9), dtype=float64, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 1., 0., ..., 1., 0., 0.]]])>,
 'policy_info': (),
 'reward': <tf.Tensor: shape=(1, 6488), dtype=float32, numpy=array([[ 0.1,  0.1, -1. , ..., -1. ,  0.1, -1. ]], dtype=float32)>,
 'step_type': <tf.Tensor: shape=(1, 6488), dtype=int32, numpy=array([[0, 1, 1, ..., 2, 2, 1]], dtype=int32)>})

In [ ]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [ ]:
iterator = iter(dataset)

In [ ]:
num_iterations = 1_000_000   # less intelligence, more persistance; 24x7 player
save_interval = 50_000
eval_interval = 10_000
log_interval = 10_000

In [ ]:
# Create a driver to collect experience.
collect_driver = DynamicStepDriver(
    tictacplayer1_tf,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=4) # collect 4 steps for each training iteration

In [ ]:
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
collect_driver.run = common.function(collect_driver.run)
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
no_of_wins, no_of_losses, no_of_ties, avg_steps_to_win, avg_steps_to_loss = compute_avg_return(tictacplayer2_tf, agent.policy, num_episodes=1)
no_of_wins_arr = np.array([no_of_wins])
no_of_losses_arr = np.array([no_of_losses])
no_of_ties_arr = np.array([no_of_ties])
avg_steps_to_win_arr = np.array([avg_steps_to_win])
avg_steps_to_loss_arr = np.array([avg_steps_to_loss])

# Reset the environment.
time_step = None
policy_state = agent.collect_policy.get_initial_state(tictacplayer1_tf.batch_size)

In [ ]:
no_of_wins_arr

array([0])

In [ ]:
no_of_losses_arr

array([1])

In [ ]:
no_of_ties_arr

array([0])

In [ ]:
import tempfile
import os
basedir = '/content/drive/My Drive/Colab Notebooks/ReinforcementLearning/TicTacToe'

In [ ]:
basedir

'/content/drive/My Drive/Colab Notebooks/ReinforcementLearning/TicTacToe'

In [ ]:
import tensorflow_probability
from tf_agents.policies import policy_saver
tf_policy_saver = policy_saver.PolicySaver(agent.policy)

In [ ]:
policy_dir = os.path.join(basedir, 'policy')

In [ ]:
policy_dir

'/content/drive/My Drive/Colab Notebooks/ReinforcementLearning/TicTacToe/policy'

In [ ]:
num_iterations = 1_000_000
while True:
    # Collect a few steps using collect_policy and save to the replay buffer.
    time_step, policy_state = collect_driver.run(time_step, policy_state)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()
    print(f'\r step {step}', end='')

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
        no_of_wins, no_of_losses, no_of_ties, avg_steps_to_win, avg_steps_to_loss = compute_avg_return(tictacplayer2_tf, agent.policy, num_episodes=25)
        print('****************************************************')
        print('step = {0}: # of wins = {1}'.format(step, no_of_wins))
        print('# of losses = {1}'.format(step, no_of_losses))
        print('# of ties = {1}'.format(step, no_of_ties))
        print('Avg steps to win = {1}'.format(step, avg_steps_to_win))
        print('Avg steps to loss = {1}'.format(step, avg_steps_to_loss))

        no_of_wins_arr = np.append(no_of_wins_arr, no_of_wins)
        no_of_losses_arr = np.append(no_of_losses_arr, no_of_losses)
        no_of_ties_arr = np.append(no_of_ties_arr, no_of_ties)
        avg_steps_to_win_arr = np.append(avg_steps_to_win_arr, avg_steps_to_win)
        avg_steps_to_loss_arr = np.append(avg_steps_to_loss_arr, avg_steps_to_loss)
        #train_checkpointer.save(global_step)

    # if step % save_interval == 0:
    #     save_checkpoint_to_local()

    if step > num_iterations:
        break

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
 step 10000step = 10000: loss = 0.12603648006916046
****************************************************
step = 10000: # of wins = 13
# of losses = 6
# of ties = 6
Avg steps to win = 4.461538461538462
Avg steps to loss = 3.8333333333333335
 step 20000step = 20000: loss = 0.1754954606294632
****************************************************
step = 20000: # of wins = 15
# of losses = 2
# of ties = 8
Avg steps to win = 4.2
Avg steps to loss = 3.5
 step 30000step = 30000: loss = 0.2097139060497284
****************************************************
step = 30000: # of wins = 18
# of losses = 3
# of ties = 4
Avg steps to win = 3.388888888888889
Avg steps to loss = 4.0
 step 40000step = 40000: loss = 0.11431829631328583
****************************************

In [ ]:
tf_policy_saver.save(policy_dir)

In [ ]:
saved_policy = tf.saved_model.load(policy_dir)

In [ ]:
action_step = saved_policy.action(time_step)

In [ ]:
action_step.action

In [ ]:
compute_avg_return(tictacplayer2_tf, saved_policy, num_episodes=100)

In [ ]:
no_of_wins_arr

In [ ]:
import matplotlib.pyplot as plt
 
# data to be plotted
x = np.arange(1, 12)
 
# plotting
plt.title("Line graph")
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.plot(x, no_of_wins_arr, color ="green")
plt.plot(x, no_of_losses_arr, color ="blue")
plt.plot(x, no_of_ties_arr, color ="yellow")
plt.show()

In [ ]:
def generatePositions(size):
        # initialize N
        # All possible N combination tuples
        # Using list comprehension + product()
        res = [ele for ele in product(range(0, size), repeat=size-1)]
        return res

In [ ]:
def showBoard(board):
    # p1: x  p2: o
    for i in range(0, 3):
        print('-------------')
        out = '| '
        for j in range(0, 3):
            if board[i, j] == 1:
                token = 'x'
            if board[i, j] == -1:
                token = 'o'
            if board[i, j] == 0:
                token = ' '
            out += token + ' | '
        print(out)
    print('-------------')

In [ ]:
def updateAction(board, action):
    board[position] = -1
    return board

In [ ]:
def get_availablePositions(board):
        positions = []
        for i in range(board.shape[0]):
            for j in range(board.shape[1]):
                if board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions


def checkGameStatus(board):
        youWinMsg = "You Win!!!!"
        youLoseMsg = "You Lose :-("
        itsTieMsg = "Its a Tie !!!"
        # row
        for i in range(board.shape[0]):
            if sum(board[i, :]) == 3:                
                return youLoseMsg
            if sum(board[i, :]) == -3:                
                return youWinMsg
        # col
        for i in range(board.shape[1]):
            if sum(board[:, i]) == 3:                
                return youLoseMsg
            if sum(board[:, i]) == -3:              
                return youWinMsg
        # diagonal
        diag_sum1 = sum([board[i, i] for i in range(board.shape[1])])
        diag_sum2 = sum([board[i, board_cols - i - 1] for i in range(board.shape[1])])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:            
            return youLoseMsg
        if diag_sum == -3:
            return youWinMsg

        # tie
        # no available positions
        if len(get_availablePositions(board)) == 0:
            return itsTieMsg
        # not end
        return 0

In [ ]:
def genTimeStep(state):
  return ts.transition(state, reward=1, discount=1.0)

In [ ]:
import numpy as np
board = np.zeros((3, 3))
base_positions = generatePositions(3)
showBoard(board)
contFlag=True

NameError: ignored

In [ ]:
while contFlag:
    row = int(input("Input your action row:"))
    col = int(input("Input your action col:"))
    user_position = (row, col)
    board = updateAction(board, user_position, -1)
    gameStatus = checkGameStatus(board)
    if gameStatus !=0:
      print(gameStatus)
      contFlag=0
    else:
      upd_state = board.flatten()
      ts_state = genTimeStep(upd_state)
      action_step = policy.action(ts_state)
      ag_action = action_step.action
      ag_position = base_positions[ag_action]
      board = updateAction(board, ag_position, 1)
      showBoard(board)